# interpolation to healpix

**Note**: Use `pixi run jupyterlab` to start a jupyterlab instance from within the project.

The UTM tile comes with a affine transform, which we can use to simplify the transformation.

For others, like `conditions/geometry` which doesn't have an affine transform, and `conditions/meteorology`, which is not UTM, we'll have to use a different approach (potentially just nearest neighbour interpolation while keeping the original coordinates).

In [ ]:
import pystac_client
import xarray as xr
import xdggs

import legacy_converters

xr.set_options(keep_attrs=True, display_expand_attrs=False, use_opt_einsum=True)

In [ ]:
from distributed import Client

client = Client()
client

## query the datasets

In [ ]:
# Access cloud-optimized Sentinel-2 data via the EOPF STAC catalog
catalog = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")

# Define oceanographic study area and time window
LON, LAT = -4.5, 48  # Bay of Biscay - known for consistent wave patterns
date = "2025-06-17/2025-06-17"

# Search criteria optimized for wave analysis

collection = "sentinel-2-l1c"
items = list(
    catalog.search(
        datetime=date,
        collections=[collection],
        intersects=dict(type="Point", coordinates=[LON, LAT]),
        query={
            "eo:cloud_cover": {
                "lt": 20
            },  # Cloud cover < 20% ensures clear ocean surface
            "view:sun_elevation": {
                "gt": 25
            },  # Filter for high sun elevation > 25° (→ sun zenith angle < 65°),
            # which places the sun near the zenith.
        },
    ).items()
)

for item in items:
    print(f"✅ {item.id}")

item = items[0]

## open the dataset

In [ ]:
dt = xr.open_datatree(
    item.assets["product"].href, engine="zarr", chunks={}, decode_timedelta=True
)
dt

## subset to a region of interest

Create raster indexes (from `rasterix`). This will allow us to keep the affine transform up-to-date, even after the subsetting.

In [ ]:
raster_dt = dt.grid4earth.create_raster_indexes()

In [ ]:
small_dt = raster_dt.sel(
    {
        "x": slice(
            dt["conditions/geometry/x"][0],
            dt["conditions/geometry/x"][1],
        ),
        "y": slice(
            dt["conditions/geometry/y"][0],
            dt["conditions/geometry/y"][1],
        ),
    }
)
small_dt

## Interpolate to healpix

Select the dataset to interpolate

In [ ]:
ds = small_dt["/measurements/reflectance/r10m"].to_dataset()
ds

infer the target grid.

Note: the method on `grid4earth` handles ellipsoids, while `xdggs` does not (yet)

In [ ]:
%%time
grid_info = xdggs.HealpixInfo(level=19, indexing_scheme="nested")
target_grid = ds.grid4earth.infer_healpix_grid(grid_info)
target_grid

### Nearest neighbour interpolation

Compute interpolation weights

Note: this is very fast because we're using a functional representation of the source grid (a affine transform) to compute neighbours.

In [ ]:
%%time
weights = legacy_converters.interpolation.weights.nearest_affine(ds, target_grid)
weights

Apply the interpolation with a sparse matrix multiplication:

In [ ]:
%%time
regridded = ds.map(
    lambda var: xr.dot(
        weights.chunk({"cells": 2000000}).variable, var, dim=["x", "y"]
    ).assign_coords(weights.dggs.coord.coords)
)
regridded

Actually compute the results:

In [ ]:
%%time
computed = regridded.compute().map(lambda var: var.copy(data=var.data.todense()))
computed

Display the result:

Note: `xdggs` does not yet support ellipsoidal coordinates, hence the shift compared to the base map

In [ ]:
computed["b03"].dggs.explore(alpha=0.8)

### Bilinear interpolation

Compute interpolation weights

Note: this is very fast because we're using a functional representation of the source grid (a affine transform) to compute neighbours.

Further improvements in the future:
- boundary treatment
- optimizations for sparse matrix-vector products

In [ ]:
%%time
weights = legacy_converters.interpolation.weights.bilinear_affine(ds, target_grid)
weights

Apply the interpolation with a sparse matrix multiplication:

In [ ]:
%%time
regridded = ds.map(
    lambda var: xr.dot(
        weights.chunk({"cells": 2000000}).variable, var, dim=["x", "y"]
    ).assign_coords(weights.dggs.coord.coords)
)
regridded

Actually compute the results:

In [ ]:
%%time
computed = regridded.compute().map(lambda var: var.copy(data=var.data.todense()))
computed

Display the result:

Note: `xdggs` does not yet support ellipsoidal coordinates, hence the shift compared to the base map

In [ ]:
computed["b03"].dggs.explore(alpha=0.8)